# <center>Breast cancer incidence rate prediction</center>

## I- Preprocessing: data importation and imputation

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
import seaborn as sns

In [ ]:
df = pd.read_csv('set_prov.csv', sep=';', decimal=',')
# Remove two columns: 'pays' and 'identifiants de pays' 
df2 = df.drop(['id','pays'],  axis='columns')
df2.shape

In [ ]:
from sklearn.impute import KNNImputer
imp_mean = KNNImputer(n_neighbors = 9)
imp_mean.fit(df2)
imputed_df1 = imp_mean.transform(df2)
imputed_df1 = pd.DataFrame(imputed_df1, columns=df2.columns)
imputed_df = imputed_df1.drop(['crude'],  axis='columns')

## II- Breast cancer incidence rates over countries in 2018

In [ ]:
quantiles = df["crude"].quantile([0.25, 0.50, 0.75])
Q1 = df[df["crude"] < quantiles[0.25]]["pays"].tolist()
Q2 = df[(df["crude"] >= quantiles[0.25]) & (df["crude"] < quantiles[0.50])]["pays"].tolist()
Q3 = df[(df["crude"] >= quantiles[0.50]) & (df["crude"] < quantiles[0.75])]["pays"].tolist()
Q4 = df[df["crude"] >= quantiles[0.75]]["pays"].tolist()

### Countries with low incidence

In [ ]:
Q1

### Countries with medium incidence

In [ ]:
Q2

### Countries with high incidence

In [ ]:
Q3

### Countries with very high incidence

In [ ]:
Q4

# III- Correlation between breast cancer incidence rates and the studied factors

In [ ]:
from scipy.stats import spearmanr, shapiro, probplot

Risk_factors_list = imputed_df.columns.tolist()
data2 = imputed_df1['crude']

spearman_corr = []
spearman_pvalue = []
normality_results = []

# Calculate number of rows needed
n_factors = len(Risk_factors_list)
n_cols = 4  # 2 histograms and 2 Q-Q plots per row
n_rows = (n_factors + (n_cols // 2) - 1) // (n_cols // 2)

# Create a figure to store all distributions
data_distribution = plt.figure(figsize=(20, n_rows * 5))

# Visualize data and evaluate normality
for i, s in enumerate(Risk_factors_list):
    data1 = imputed_df[s]
    
    # Normality check
    shapiro_test = shapiro(data1)
    normality_results.append({
        'Factor': s,
        'Statistic': shapiro_test.statistic,
        'P-value': shapiro_test.pvalue,
        'Normal Distribution': shapiro_test.pvalue > 0.05
    })
    
    # Add histogram
    plt.subplot(n_rows, n_cols, i * 2 + 1)
    sns.histplot(data1, kde=True)
    plt.title(f'Histogram of {s}')
    
    # Add Q-Q plot
    plt.subplot(n_rows, n_cols, i * 2 + 2)
    probplot(data1, dist="norm", plot=plt)
    plt.title(f'Q-Q Plot of {s}')

    # Calculate Spearman's rank correlation
    rho, p = spearmanr(data1, data2)
    spearman_corr.append(np.around(rho, 2))
    spearman_pvalue.append(p)

# Create DataFrames for normality and Spearman results
normality_results_df = pd.DataFrame(normality_results)
spearman_result_table = pd.DataFrame({
    "Correlation": spearman_corr,
    "P-value": spearman_pvalue,
}, index=Risk_factors_list)

# Save the figure
data_distribution.savefig('data_distribution.png', bbox_inches='tight')

# Print the results
print("Shapiro-Wilk Test Results:")
print(normality_results_df)

print("\nSpearman Correlation Results:")
spearman_result_table.sort_values(by='Correlation', ascending=False, inplace=True)
print(spearman_result_table)

## IV- Breast Cancer Incidence Rate Prediction

#### Spliting data to X and Y and standardizing X set

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = imputed_df1.drop(['crude'], axis = 1)
Y = imputed_df1.crude
X = StandardScaler().fit_transform(X)

#### Spliting X and Y to train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)

#### Importing regressors

In [ ]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE

#### Forward Feature Selection for each regressor

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
result_for_each_num_of_features = []

regressors = [SVR(kernel = 'linear'),
              LinearRegression(),
              RandomForestRegressor(),
              XGBRegressor(),
              LGBMRegressor(),
              CatBoostRegressor(silent = True)]
reg_names = ['SVR','Linear Reg','Random Forest','XGBoost', 'Light GBM', 'CatBoost']
for regressor, r_name in zip(regressors, reg_names):
    result_for_each_num_of_features.clear()
    print('='*20,end="")
    print('\t', r_name, '\t', end="")
    print('='*20)
    
    for i in range(1,16):
        # Testing FFS        
        # Build step forward feature selection
        sfs1 = sfs(regressor,
                   k_features = i, 
                   forward = True,
                   floating = False,
                   verbose = 2,
                   scoring = 'r2',
                   cv = 5,
                   )
        # Perform SFFS
        sfs1 = sfs1.fit(X_train, y_train)
        # Which features?
        feat_cols = list(sfs1.k_feature_idx_)
        # Build full model with selected features
        regressor.fit(X_train[:, feat_cols], y_train)
        y_test_pred = regressor.predict(X_test[:, feat_cols])
        result_for_each_num_of_features.append(r2_score(y_test, y_test_pred))
    print('-'*90)
    print('='*20,end="")
    print('\t', r_name, '\t', end="")
    print('='*20)
    for result in result_for_each_num_of_features:
        print("%.3f" % result)  
    print('-'*90)

In [ ]:
import random
randomlist = []
for i in range(0,50):
    n = random.randint(2000,2500)
    randomlist.append(n)

### SVR

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)
feat_cols = []
# Testing FFS
# Build Regressor to use in feature selection
reg = SVR(kernel = 'linear')
# Build step forward feature selection
sfs1 = sfs(reg,
           k_features = 8, # choisir après constatation
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 5,)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
# Build full model with selected features
reg.fit(X_train[:, feat_cols], y_train)
y_test_pred = reg.predict(X_test[:, feat_cols])
print('Testing R² on selected features: %.3f' % r2_score(y_test, y_test_pred))

In [ ]:
r_squ = []
MAEe = []
RMSEe = []

regressor = SVR(kernel = 'linear')
for i in randomlist:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = i)
    regressor.fit(X_train[:, feat_cols], y_train)
    preds = regressor.predict(X_test[:, feat_cols])
    r_squ.append(r2_score(y_test, preds))
    MAEe.append(MAE(y_test, preds))
    RMSEe.append(np.sqrt(MSE(y_test, preds)))
print("Mean R squared")
print("%.3f" % np.mean(r_squ), "±",  "%.3f" % np.std(r_squ))
print("Mean MAE")
print("%.3f" % np.mean(MAEe), "±",  "%.3f" % np.std(MAEe))
print("Mean RMSE")
print("%.3f" % np.mean(RMSEe), "±",  "%.3f" % np.std(RMSEe))

### Linear Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)
feat_cols = []
# Testing FFS
# Build Regressor to use in feature selection
reg = LinearRegression()
# Build step forward feature selection
sfs1 = sfs(reg,
           k_features = 2, # choisir après constatation
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 5,)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
# Build full model with selected features
reg.fit(X_train[:, feat_cols], y_train)
y_test_pred = reg.predict(X_test[:, feat_cols])
print('Testing R² on selected features: %.3f' % r2_score(y_test, y_test_pred))

In [ ]:
r_squ = []
MAEe = []
RMSEe = []

regressor = LinearRegression()
for i in randomlist:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = i)
    regressor.fit(X_train[:, feat_cols], y_train)
    preds = regressor.predict(X_test[:, feat_cols])
    r_squ.append(r2_score(y_test, preds))
    MAEe.append(MAE(y_test, preds))
    RMSEe.append(np.sqrt(MSE(y_test, preds)))
print("Mean R squared")
print("%.3f" % np.mean(r_squ), "+",  "%.3f" % np.std(r_squ))
print("Mean MAE")
print("%.3f" % np.mean(MAEe), "+",  "%.3f" % np.std(MAEe))
print("Mean RMSE")
print("%.3f" % np.mean(RMSEe), "+",  "%.3f" % np.std(RMSEe))

### Random Forest 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)
feat_cols = []
# Testing FFS
# Build Regressor to use in feature selection
reg = RandomForestRegressor()
# Build step forward feature selection
sfs1 = sfs(reg,
           k_features = 10, # choisir après constatation
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 5,)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
# Build full model with selected features
reg.fit(X_train[:, feat_cols], y_train)
y_test_pred = reg.predict(X_test[:, feat_cols])
print('Testing R² on selected features: %.3f' % r2_score(y_test, y_test_pred))

In [ ]:
r_squ = []
MAEe = []
RMSEe = []

regressor = RandomForestRegressor()
for i in randomlist:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = i)
    regressor.fit(X_train[:, feat_cols], y_train)
    preds = regressor.predict(X_test[:, feat_cols])
    r_squ.append(r2_score(y_test, preds))
    MAEe.append(MAE(y_test, preds))
    RMSEe.append(np.sqrt(MSE(y_test, preds)))
print("Mean R squared")
print("%.3f" % np.mean(r_squ), "+",  "%.3f" % np.std(r_squ))
print("Mean MAE")
print("%.3f" % np.mean(MAEe), "+",  "%.3f" % np.std(MAEe))
print("Mean RMSE")
print("%.3f" % np.mean(RMSEe), "+",  "%.3f" % np.std(RMSEe))

### XGBoost

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)
feat_cols = []
# Testing FFS
# Build Regressor to use in feature selection
reg = XGBRegressor()
# Build step forward feature selection
sfs1 = sfs(reg,
           k_features = 15, # choisir après constatation 
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 5,)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
# Build full model with selected features
reg.fit(X_train[:, feat_cols], y_train)
y_test_pred = reg.predict(X_test[:, feat_cols])
print('Testing R² on selected features: %.3f' % r2_score(y_test, y_test_pred))

In [ ]:
r_squ = []
MAEe = []
RMSEe = []

regressor = XGBRegressor()
for i in randomlist:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = i)
    regressor.fit(X_train[:, feat_cols], y_train)
    preds = regressor.predict(X_test[:, feat_cols])
    r_squ.append(r2_score(y_test, preds))
    MAEe.append(MAE(y_test, preds))
    RMSEe.append(np.sqrt(MSE(y_test, preds)))
print("Mean R squared")
print("%.3f" % np.mean(r_squ), "+",  "%.3f" % np.std(r_squ))
print("Mean MAE")
print("%.3f" % np.mean(MAEe), "+",  "%.3f" % np.std(MAEe))
print("Mean RMSE")
print("%.3f" % np.mean(RMSEe), "+",  "%.3f" % np.std(RMSEe))

### LGBMRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)
feat_cols = []
# Testing FFS
# Build Regressor to use in feature selection
reg = LGBMRegressor()
# Build step forward feature selection
sfs1 = sfs(reg,
           k_features = 10, # choisir après constatation
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 5,)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
# Build full model with selected features
reg.fit(X_train[:, feat_cols], y_train)
y_test_pred = reg.predict(X_test[:, feat_cols])
print('Testing R² on selected features: %.3f' % r2_score(y_test, y_test_pred))

In [ ]:
r_squ = []
MAEe = []
RMSEe = []

regressor = LGBMRegressor()
for i in randomlist:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = i)
    regressor.fit(X_train[:, feat_cols], y_train)
    preds = regressor.predict(X_test[:, feat_cols])
    r_squ.append(r2_score(y_test, preds))
    MAEe.append(MAE(y_test, preds))
    RMSEe.append(np.sqrt(MSE(y_test, preds)))
print("Mean R squared")
print("%.3f" % np.mean(r_squ), "+",  "%.3f" % np.std(r_squ))
print("Mean MAE")
print("%.3f" % np.mean(MAEe), "+",  "%.3f" % np.std(MAEe))
print("Mean RMSE")
print("%.3f" % np.mean(RMSEe), "+",  "%.3f" % np.std(RMSEe))

### CatBoost

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 2276)
feat_cols = []
# Testing FFS
# Build Regressor to use in feature selection
reg = CatBoostRegressor(silent = True)
# Build step forward feature selection
sfs1 = sfs(reg,
           k_features = 13, # choisir après constatation
           forward = True,
           floating = False,
           verbose = 2,
           scoring = 'r2',
           cv = 5,)

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)
# Build full model with selected features
reg.fit(X_train[:, feat_cols], y_train)
y_test_pred = reg.predict(X_test[:, feat_cols])
print('Testing R² on selected features: %.3f' % r2_score(y_test, y_test_pred))

In [ ]:
r_squ = []
MAEe = []
RMSEe = []

regressor = CatBoostRegressor(silent = True)
for i in randomlist:
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = i)
    regressor.fit(X_train[:, feat_cols], y_train)
    preds = regressor.predict(X_test[:, feat_cols])
    r_squ.append(r2_score(y_test, preds))
    MAEe.append(MAE(y_test, preds))
    RMSEe.append(np.sqrt(MSE(y_test, preds)))
print("Mean R squared")
print("%.3f" % np.mean(r_squ), "+",  "%.3f" % np.std(r_squ))
print("Mean MAE")
print("%.3f" % np.mean(MAEe), "+",  "%.3f" % np.std(MAEe))
print("Mean RMSE")
print("%.3f" % np.mean(RMSEe), "+",  "%.3f" % np.std(RMSEe))

### Features importance for the best regressor (CatBoostRegressor)

In [ ]:
for i in range(0,16):
    print(i,'\t',imputed_df.columns[i])

In [ ]:
my_data = imputed_df.drop(['crude'], axis = 1)
my_data.columns

reg = CatBoostRegressor(silent = True)
reg.fit(X_train, y_train)
sort = reg.feature_importances_.argsort()
plt.barh(my_data.columns[sort], reg.feature_importances_[sort])
plt.xlabel('Importance score', fontsize = 10)
plt.ylabel('Feature', fontsize = 10, rotation = 90)
plt.tight_layout()
plt.savefig('importance.png', dpi = 300)
plt.show()